In [22]:
import seaborn as sns
import torch
import torch.nn as nn

# PEGANDO MENOS REGISTROS PORQUE SE NÃO MEU PC NÃO AGUENTA
diamonds = sns.load_dataset("diamonds").head(10000)

print('Tem algum nulo? ' + str(diamonds.isna().any().any()))

map_cut = {'Fair':0,'Good':1,'Very Good':2,'Premium':3,'Ideal':4}
map_color = {'J':0,'I':1,'H':2,'G':3,'F':4,'E':5,'D':6}
map_clarity = {'I1':0,'SI2':1,'SI1':2,'VS2':3,'VS1':4,'VVS2':5,'VVS1':6,'IF':7}

train=diamonds.sample(frac=0.8,random_state=42)
test=diamonds.drop(train.index)

train["cut"] = train.cut.map(map_cut)
test["cut"]  = test.cut.map(map_cut)

train["color"] = train.color.map(map_color)
test["color"]  = test.color.map(map_color)

train["clarity"] = train.clarity.map(map_clarity)
test["clarity"]  = test.clarity.map(map_clarity)

x_train = train.loc[ : ,  ["carat","cut","color","clarity","depth","table","x","y","z"]].values
x_test  = test.loc[ : , ["carat","cut","color","clarity","depth","table","x","y","z"]].values

y_train = train.price.values
y_test  = test.price.values

x_train = torch.tensor(x_train, dtype = torch.float)
x_test  = torch.tensor(x_test, dtype = torch.float)

y_train = torch.tensor(y_train, dtype = torch.float)
y_test  = torch.tensor(y_test, dtype = torch.float)


x_train = (x_train - x_train.min())/(x_train.max() - x_train.min())
x_test  = (x_test  - x_test.min())/(x_test.max() - x_test.min())

y_train = (y_train - y_train.min())/(y_train.max() - y_train.min())
y_test  = (y_test  - y_test.min())/(y_test.max() - y_test.min())


class NeuralNetwork(nn.Module):
    def __init__(self, input_dim):
        super(NeuralNetwork, self).__init__()
        self.input_layer = nn.Linear(input_dim, 128)
        self.hidden_layer1 = nn.Linear(128, 64)
        self.output_layer = nn.Linear(64, 1)
        
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu(self.input_layer(x))
        out = self.relu(self.hidden_layer1(out))
        return self.output_layer(out)

input_dim = x_train.shape[1]
model = NeuralNetwork(input_dim)

loss_fn = nn.MSELoss()

learning_rate = 0.001 
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

epochs = 10000

batch_size = 1000
batch_no = len(x_train) // batch_size

for epoch in range(epochs):
    for i in range(batch_no):
        start = i * batch_size
        end = start + batch_size

        x_var = x_train[start:end]
        y_var = y_train[start:end]

        optimizer.zero_grad()

        output_train = model(x_var).squeeze(-1)

        loss = loss_fn(output_train, y_var)

        if (epoch % (epochs / 10) == 0 and i == 0) :
            print(epoch, loss.item())

        loss.backward()
        optimizer.step()

y_pred = model(x_test).squeeze(-1)

with open('y_pred.txt', 'w') as f:
    f.write(str(y_pred[:100]))

with open('y_test.txt', 'w') as f:
    f.write(str(y_test[:100]))

medio = torch.mean(torch.abs(y_pred.squeeze() - y_test))
print('Erro Médio Absoluto:', medio.item())

porcentagem = 100 - ((medio.item() / torch.mean(torch.abs(y_test)).item()) * 100)
print('Porcentagem de Acerto:', porcentagem)

# melhor resultado: 90% de acerto


Tem algum nulo? False
0 0.6509931087493896
1000 0.004778883419930935
2000 0.0042469254694879055
3000 0.0041318717412650585
4000 0.003953306004405022
5000 0.0038963279221206903
6000 0.0038446050602942705
7000 0.0037886002101004124
8000 0.0037730252370238304
9000 0.0037481042090803385
Erro Médio Absoluto: 0.06343850493431091
Porcentagem de Acerto: 90.92163309040968
